In [1]:
!pip install yfinance

     |████████████████████████████████| 5.5MB 6.5MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=5f395a45d54412ca092a06db18970b6aae3d67430312425d6d54637c4db7538c
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import pandas as pd  
from sklearn.model_selection import train_test_split 
from sklearn.neural_network import MLPRegressor
from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, cross_validate, KFold
from numpy import mean, absolute, sqrt


In [3]:
stocks = ["SPY"]
spy_yf = yf.download(stocks, start="2000-01-01", end="2021-02-20")
spy = pd.DataFrame(spy_yf)


[*********************100%***********************]  1 of 1 completed


In [4]:
# spy.to_csv('SPY.csv', index=False)

In [5]:
spy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5317 entries, 2000-01-03 to 2021-02-19
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       5317 non-null   float64
 1   High       5317 non-null   float64
 2   Low        5317 non-null   float64
 3   Close      5317 non-null   float64
 4   Adj Close  5317 non-null   float64
 5   Volume     5317 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 290.8 KB


In [6]:
spy.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [7]:
spy_x = spy.drop(['Close', 'Adj Close'], axis =1)
spy_y = spy['Close'] # Predict closing price

In [8]:
spy_x_train, spy_x_test, spy_y_train, spy_y_test = train_test_split(spy_x, spy_y, test_size=0.3) # Simple Split

In [9]:
mlpRegression = MLPRegressor(activation ='logistic', hidden_layer_sizes=(20,), max_iter= 3000, solver = 'sgd', random_state=2)
mlpRegression.fit(spy_x_train, spy_y_train)

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(20,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=3000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=2, shuffle=True, solver='sgd',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [10]:
y_train_predict = mlpRegression.predict(spy_x_train) # Store the predictions
y_test_predict = mlpRegression.predict(spy_x_test)

In [11]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)

results = cross_val_score(mlpRegression, spy_x, spy_y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
sqrt(mean(absolute(results)))
# results

69.14212360493153

In [12]:
print("Artificial Neural Network")
print("Cross Validation RSME:", sqrt(mean(absolute(results))))

print("")

print('Training R2 Value:', mlpRegression.score(spy_x_train, spy_y_train))
print('Testing R2 Value:', mlpRegression.score(spy_x_test, spy_y_test))

print("")

print('Training RMSE:', sqrt(mean_squared_error(y_train_predict, spy_y_train))) # RMSE: the lower the better
print('Testing RMSE:', sqrt(mean_squared_error(y_test_predict, spy_y_test))) # RMSE: the lower the better

# Before XGB
# Artificial Neural Network
# Training R2 Value: -6.139917898995151e-06
# Testing R2 Value: -0.000464315194173448

# Training RMSE: 68.78907552942923
# Testing RMSE: 69.93273864746622

Artificial Neural Network
Cross Validation RSME: 69.14212360493153

Training R2 Value: -2.7991139739391713e-06
Testing R2 Value: -0.003197903914230027

Training RMSE: 69.5406126480552
Testing RMSE: 68.19800205251903


In [13]:
# param_grid1 = {"activation": ["logistic", "relu"], "hidden_layer_sizes" : [(10,), (15,), (20,)], "max_iter" : [500, 1000, 2000, 3000], 'solver' : ["sgd", "adam", "lgbfs"]}

# spyGS = GridSearchCV(mlpRegression, param_grid=param_grid1, cv=15, n_jobs=-1)

# spyGS = spyGS.fit(spy_x_train, spy_y_train) 

# print(spyGS.best_score_)
# print(spyGS.best_params_)

In [14]:
mlpRegression_Improved = MLPRegressor(activation ='logistic', hidden_layer_sizes=(20,), max_iter= 500, solver = 'sgd', random_state=2)
mlpRegression_Improved.fit(spy_x_train, spy_y_train)

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(20,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=500,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=2, shuffle=True, solver='sgd',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [15]:
cv_GS = KFold(n_splits=10, random_state=1, shuffle=True)

results_GS = cross_val_score(mlpRegression_Improved, spy_x, spy_y, scoring='neg_mean_squared_error', cv=cv_GS, n_jobs=-1)
sqrt(mean(absolute(results_GS)))

69.14212360493153

In [16]:
print('Artificial Neural Network')
print('Training R^2 Value:', mlpRegression.score(spy_x_train, spy_y_train), '\tImproved Training R^2 Value:', mlpRegression_Improved.score(spy_x_train, spy_y_train))
print('Testing R^2 Value:', mlpRegression.score(spy_x_test, spy_y_test), '\tImproved Testing R^2 Value:', mlpRegression_Improved.score(spy_x_test, spy_y_test))

# Artificial Neural Network
# Training R^2 Value: -6.139917898995151e-06 	Improved Training R^2 Value: 0.05402620250393664
# Testing R^2 Value: -0.000464315194173448 	Improved Testing R^2 Value: 0.08688677788291521

# After XGB
# Training R^2 Value: -3.2053089826789005e-06 	Improved Training R^2 Value: 0.9819630816690547
# Testing R^2 Value: -0.0006414591987917717 	Improved Testing R^2 Value: 0.9815814374859191

Artificial Neural Network
Training R^2 Value: -2.7991139739391713e-06 	Improved Training R^2 Value: -2.7991139739391713e-06
Testing R^2 Value: -0.003197903914230027 	Improved Testing R^2 Value: -0.003197903914230027


In [17]:
y_train_improved_predict = mlpRegression_Improved.predict(spy_x_train) # Store the predictions using the improved model
y_test_improved_predict = mlpRegression_Improved.predict(spy_x_test)

In [18]:
print("Artificial Neural Network")
print('Training R2 Value:', mlpRegression_Improved.score(spy_x_train, spy_y_train)) 
print('Testing R2 Value:', mlpRegression_Improved.score(spy_x_test, spy_y_test))

print("")


print('Training RMSE:', sqrt(mean_squared_error(y_train_improved_predict, spy_y_train))) # RMSE: the lower the better
print('Testing RMSE:', sqrt(mean_squared_error(y_test_improved_predict, spy_y_test))) # RMSE: the lower the better

# Artificial Neural Network
# Training R2 Value: 0.05402620250393664
# Testing R2 Value: 0.08688677788291521

# Training RMSE: 66.90486414652831
# Testing RMSE: 66.81008902408266

# After XGB

# Training R2 Value: 0.9819630816690547
# Testing R2 Value: 0.9815814374859191

# Training RMSE: 9.194656855492699
# Testing RMSE: 9.4776522763638

Artificial Neural Network
Training R2 Value: -2.7991139739391713e-06
Testing R2 Value: -0.003197903914230027

Training RMSE: 69.5406126480552
Testing RMSE: 68.19800205251903


In [19]:
# pred = mlpRegression_Improved.predict(spy_x)
# spy['Predicted'] = pred

In [20]:
spy

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,148.250000,148.250000,143.875000,145.437500,98.146072,8164300
2000-01-04,143.531250,144.062500,139.640625,139.750000,94.307983,8089800
2000-01-05,139.937500,141.531250,137.250000,140.000000,94.476685,12177900
2000-01-06,139.625000,141.500000,137.750000,137.750000,92.958336,6227200
2000-01-07,140.312500,145.750000,140.062500,145.750000,98.356987,8066500
...,...,...,...,...,...,...
2021-02-12,389.850006,392.899994,389.769989,392.640015,392.640015,50505700
2021-02-16,393.959991,394.170013,391.529999,392.299988,392.299988,50700800
2021-02-17,390.420013,392.660004,389.329987,392.390015,392.390015,52290600


In [21]:
#XGBoost
!pip install xgboost

In [22]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error

In [23]:
# X, y = spy.iloc[:,:-1],spy.iloc[:,-1] 
# X = spy.drop(['Close'], axis =1)
# y = spy['Close'] # Predict closing price

# We just use SPY_X and SPY_Y can le ba no need redo this part

In [24]:
data_dmatrix = xgb.DMatrix(data=spy_x,label=spy_y)

In [25]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# We just use SPY_X's Train / Test and SPY_Y Train / Test can le ba no need redo this part

In [26]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror',
                max_depth = 25, n_estimators = 100)

In [27]:
xg_reg.fit(spy_x_train, spy_y_train)

# preds = xg_reg.predict(X_test)
preds = xg_reg.predict(spy_x)

preds_train = xg_reg.predict(spy_x_train)
preds_test = xg_reg.predict(spy_x_test)

In [28]:
cv_XGB = KFold(n_splits=10, random_state=1, shuffle=True)

results_XGB = cross_val_score(xg_reg, spy_x, spy_y, scoring='neg_mean_squared_error', cv=cv_XGB, n_jobs=-1)
sqrt(mean(absolute(results_XGB)))

1.0935589545441882

In [29]:
# rmse = np.sqrt(mean_squared_error(y_test, preds))
# print("RMSE: %f" % (rmse))
print("Cross Validation XGB:", sqrt(mean(absolute(results_XGB))))

print("")

print('Training R2 Value:', xg_reg.score(spy_x_train, spy_y_train)) 
print('Testing R2 Value:', xg_reg.score(spy_x_test, spy_y_test))

print("")

print('Training RMSE:', sqrt(mean_squared_error(preds_train, spy_y_train))) # RMSE: the lower the better
print('Testing RMSE:', sqrt(mean_squared_error(preds_test, spy_y_test))) # RMSE: the lower the better

# Training R2 Value: 0.9999991980659549
# Testing R2 Value: 0.9997562180800605

# Training RMSE: 0.06160095786407599
# Testing RMSE: 1.091642615309928

Cross Validation XGB: 1.0935589545441882

Training R2 Value: 0.9999991593777121
Testing R2 Value: 0.9997141548870484

Training RMSE: 0.06375853871945981
Testing RMSE: 1.1511813083359443


In [30]:
spy['XGBPredict'] = preds 

In [31]:
spy[["Open", "Close","XGBPredict"]]

,Open,Close,XGBPredict
Date,,,
2000-01-03,148.250000,145.437500,145.433929
2000-01-04,143.531250,139.750000,139.752747
2000-01-05,139.937500,140.000000,139.989853
2000-01-06,139.625000,137.750000,137.771332
2000-01-07,140.312500,145.750000,145.695572
...,...,...,...
2021-02-12,389.850006,392.640015,391.845612
2021-02-16,393.959991,392.299988,391.574341
2021-02-17,390.420013,392.390015,391.845612


In [32]:
predict = pd.DataFrame(np.array([[386.33, 392.23, 385.27, 72433946]]), columns=['Open', 'High', 'Low', 'Volume'])
xg_reg.predict(predict)

array([387.73016], dtype=float32)

In [30]:
xg_reg.save_model('SPY_Model.pkl')

In [31]:
# import pickle

# filename = 'SPY_Model.pkl'
# pickle.dump(xg_reg, open(filename, 'wb'))


In [32]:
model = pickle.load(open('SPY_Model.pkl','rb'))

NameError: ignored

In [ ]:
spy.info()